# TASK 3: Przygotowanie Zbiorów Treningowych

W tym notebooku:
1. Wczytamy dane GTZAN
2. Wyodrębnimy 59 cech z librosa
3. Podzielimy dane na train/val/test (60/20/20)
4. Przeanalizujemy cechy

## Setup

In [ ]:
import sys
from pathlib import Path

# Dodaj src do path'u
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from feature_extractor import FeatureExtractor
from dataset_splitter import DatasetSplitter

# Wizualizacja
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Setup ukończony!")

## KROK 1: Ekstrakcja 59 Cech

In [ ]:
# Ścieżki
DATA_DIR = Path.cwd().parent / 'data' / 'raw' / 'genres'
OUTPUT_DIR = Path.cwd().parent / 'data' / 'processed'

print(f"Data dir: {DATA_DIR}")
print(f"Output dir: {OUTPUT_DIR}")
print(f"\nData dir istnieje: {DATA_DIR.exists()}")

In [ ]:
# Ekstrakcja cech
print("🎵 Ekstrakcja 59 cech z GTZAN...\n")

extractor = FeatureExtractor(sr=22050)
features_df = extractor.extract_from_directory(str(DATA_DIR))

In [ ]:
# Przegląd danych
print(f"Shape: {features_df.shape}")
print(f"\nPierwsze 5 wierszy:")
features_df.head()

In [ ]:
# Informacje o cechach
print(f"Kolumny: {list(features_df.columns)}")
print(f"\nDatTypy:")
features_df.dtypes

In [ ]:
# Statystyka cech
print("📊 Statystyka cech (pierwsze 10):")
features_df.describe().iloc[:, :10]

## KROK 2: Analiza Rozkładu Gatunków

In [ ]:
# Rozkład gatunków
genre_counts = features_df['genre'].value_counts().sort_index()

print(f"🎼 Rozkład gatunków:")
print(genre_counts)
print(f"\nRazem: {genre_counts.sum()} próbek")

In [ ]:
# Wizualizacja rozkładu gatunków
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
genre_counts.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Rozkład Próbek po Gatunkach', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Gatunek')
axes[0].set_ylabel('Liczba Próbek')
axes[0].grid(axis='y', alpha=0.3)

# Pie chart
genre_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
axes[1].set_title('Procent Próbek po Gatunkach', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / 'genre_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()

print("✅ Wykres zapisany!")

## KROK 3: Podział na Train/Val/Test

In [ ]:
# Podział danych
splitter = DatasetSplitter(test_size=0.2, val_size=0.2, random_state=42)
train_df, val_df, test_df = splitter.split(features_df, label_column='genre')

In [ ]:
# Zapisz podzielone zbiory
splitter.save_splits(train_df, val_df, test_df, output_dir=str(OUTPUT_DIR))

## KROK 4: Analiza Wyodrębnionych Cech

In [ ]:
# Wybierz tylko cechy (bez genre i filename)
feature_cols = [col for col in features_df.columns if col not in ['genre', 'filename']]
print(f"Liczba cech: {len(feature_cols)}")
print(f"\nPierwsze 10 cech: {feature_cols[:10]}")

In [ ]:
# Histogram top 9 cech
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

top_features = feature_cols[:9]

for idx, feature in enumerate(top_features):
    axes[idx].hist(features_df[feature], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'{feature}', fontweight='bold')
    axes[idx].set_xlabel('Wartość')
    axes[idx].set_ylabel('Liczba Próbek')
    axes[idx].grid(axis='y', alpha=0.3)

plt.suptitle('Rozkład Top 9 Cech', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / 'feature_distributions.png'), dpi=150, bbox_inches='tight')
plt.show()

print("✅ Wykresy zapisane!")

In [ ]:
# Heatmapa korelacji top cech
fig, ax = plt.subplots(figsize=(12, 10))

top_20_features = feature_cols[:20]
corr_matrix = features_df[top_20_features].corr()

sns.heatmap(corr_matrix, cmap='coolwarm', center=0, annot=False, ax=ax, 
            square=True, linewidths=0.5, cbar_kws={'label': 'Korelacja'})
ax.set_title('Macierz Korelacji Top 20 Cech', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / 'feature_correlation.png'), dpi=150, bbox_inches='tight')
plt.show()

print("✅ Heatmapa zapisana!")

## KROK 5: Podsumowanie

In [ ]:
print("\n" + "="*70)
print("✅ TASK 3 UKOŃCZONY")
print("="*70)

print(f"\n📊 STATYSTYKA:")
print(f"\n  Cechy:")
print(f"    • Liczba cech: {len(feature_cols)}")
print(f"    • Próbki całość: {len(features_df)}")

print(f"\n  Zbiór treningowy (60%):")
print(f"    • Próbki: {len(train_df)}")
print(f"    • Plik: data/processed/train_features.csv")

print(f"\n  Zbiór walidacyjny (20%):")
print(f"    • Próbki: {len(val_df)}")
print(f"    • Plik: data/processed/val_features.csv")

print(f"\n  Zbiór testowy (20%):")
print(f"    • Próbki: {len(test_df)}")
print(f"    • Plik: data/processed/test_features.csv")

print(f"\n  Gatunki: {features_df['genre'].nunique()}")
print(f"    • {', '.join(sorted(features_df['genre'].unique()))}")

print(f"\n💡 Następny krok:")
print(f"   TASK 4: Trening kNN classifier")
print(f"   Otworz: notebooks/Task4_KNNBenchmark.ipynb")
print(f"\n" + "="*70)